In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Tokenizer, GPTJForCausalLM, DataCollatorWithPadding
from datasets import load_dataset

In [2]:
#model_type = "hf-internal-testing/tiny-random-gptj"
model_type = "gpt2-medium"
tokenizer =   AutoTokenizer.from_pretrained(model_type)

hface_model = AutoModelForCausalLM.from_pretrained(model_type)

In [3]:
#hface_model

In [4]:
#hface_model.parallelize()

In [5]:
#hface_model.deparallelize()

In [6]:
# Ignore this for now
#small_dataset = {
#    "rand": {
#        "prompts": [
#            "he's bald, he's mauled, he's called grindlewald... or so I'm tald.",
#            "To win, you must move forward at all costs. There is no retreat. I get knocked down, but I get up again. You will never keep me down.",
#            "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam", 
#            "quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum",
#            "dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
#        ],
#        "answers": [
#            "But why does the pilgrim seek refuge?",
#            "Backoff, headstrong",
#            "Sed ut perspiciatis unde omnis iste natus error",
#            "sit voluptatem accusantium doloremque laudantium",
#            "totam rem aperiam, eaque ipsa quae ab illo inventore",
#        ]
#    },
#    "blicket_direct": {
#        "prompts": [
#            "You have two items. At least one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine does not turn on. Is the item a blicket?",
#            "You have two items. At least one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the item a blicket?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fail to fix it. Is the tool you used magic?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool you used magic?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins wobbly. Is that egg hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Is that egg hardboiled?",
#        ],
#        "answers": [
#            "no, it is not",
#            "yes, it is",
#            "no, it is not",
#            "yes, it is",
#            "no, it is not",
#            "yes, it is",
#        ]
#    },
#    "blicket_implicit": {
#        "prompts": [
#            "You have two items. At least one is a blicket. If an item touches a machine and the machine turns on, the item is a blicket. You take one of the items and place it on a machine. The machine does not turn on. Is the other item a blicket?",
#            "You have two items. At least one is a blicket. If an item touches a machine and the machine turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the other item a blicket?",
#            "You have two items. Only one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the other item a blicket?",
#            
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fail to fix it. Is the tool you did not use magic?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool that you did not use magic?",
#            "You have two tools. Only one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool that you did not use magic?",
#            
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins wobbly. Are the other eggs all hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Are the other eggs all hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at only 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Are the other eggs all hardboiled?",
#        ],
#        "answers": [
#            "yes, it is",
#            "we cannot know the answer from the given information",
#            "no, it is not",
#            
#            "yes, it is",
#            "we cannot know the answer from the given information",
#            "no, it is not",
#            
#            "yes, they are",
#            "we cannot know the answer from the given information",
#            "no, they are not",
#        ]
#    }
#}
#
#string = small_dataset["rand"]["prompts"][0]
#string

In [7]:
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

## Data Manipulations

In [8]:
MAX_SEQ_LEN = 100
bsize = 16

In [9]:
dataset = load_dataset("glue", "mrpc", split="train")
dataset

Found cached dataset glue (/home/grantsrb/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [10]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [11]:
new_tokens = {
    "pad_token": "<PAD>",
    "cls_token": "<CLS>", # Using CLS token as compression token
}


In [12]:
num_added = tokenizer.add_special_tokens(new_tokens)

In [13]:
n,h = hface_model.transformer.wte.weight.shape
n

50257

In [14]:
hface_model.resize_token_embeddings(n+num_added)

Embedding(50259, 1024)

In [15]:
s = "heyo bo"
x = s[:100] + new_tokens["cls_token"]
x

'heyo bo<CLS>'

In [16]:
def encode(examples):
    sent1 = [s + tokenizer.cls_token for s in examples["sentence1"]]
    inpts = tokenizer(sent1,padding="max_length", max_length=MAX_SEQ_LEN, truncation=True, return_tensors="pt")
    sent2 = [s + tokenizer.eos_token for s in examples["sentence2"]]
    outs = tokenizer(sent2,padding="max_length", max_length=MAX_SEQ_LEN, truncation=True , return_tensors="pt")
    
    examples["label"] = torch.LongTensor(examples["label"])
    idx = examples["label"]==0
    
    outs["input_ids"][idx] = inpts["input_ids"][idx].clone()
    outs["attention_mask"][idx] = inpts["attention_mask"][idx].clone()
    outs["input_ids"][outs["input_ids"]==tokenizer.cls_token_id] = tokenizer.eos_token_id
    
    idx = (inpts["input_ids"]==tokenizer.cls_token_id).float().reshape(len(idx),-1).sum(-1)
    if idx.sum() != len(idx):
        i = torch.argmax((idx==0).float())
        inpts["input_ids"][i,-1] = tokenizer.cls_token_id
    idx = (outs["input_ids"]==tokenizer.eos_token_id).float().reshape(len(examples["label"]),-1).sum(-1)
    if idx.sum() != len(examples["label"]):
        i = torch.argmax((idx==0).float())
        outs["input_ids"][i,-1] = tokenizer.eos_token_id
    
    #s = (inpts["input_ids"]==tokenizer.cls_token_id).float().reshape(len(idx),-1).sum()
    #if s != len(inpts["input_ids"]):
    #    print("CLS token id:", tokenizer.cls_token_id)
    #    print("Sum:", s)
    #    print("Len:", len(inpts["input_ids"]))
    #    x = inpts["input_ids"]
    #    idx = (x==tokenizer.cls_token_id).float().reshape(x.shape[0],-1).sum(-1)
    #    idx = torch.argmax((idx==0).float())
    #    print("idx:", idx)
    #    print(sent1[idx])
    #    print(inpts["input_ids"][idx])
    #    print()
    #    print()
    
    return {
        "input_ids":        inpts["input_ids"],
        "attention_mask":   inpts["attention_mask"],
        "output_ids":       outs["input_ids"],
        "output_attn_mask": outs["attention_mask"],
        "labels":           examples["label"],
    }
    
dataset = dataset.map(encode, batched=True)

Loading cached processed dataset at /home/grantsrb/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3c66e6bbed442bd8.arrow


In [17]:
dataset[0].keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask', 'output_ids', 'output_attn_mask', 'labels'])

In [18]:
len(dataset["input_ids"][0])

100

In [19]:
len(dataset["output_ids"][0])

100

In [20]:
dataset = dataset.remove_columns(["sentence1", "sentence2", "idx", "label"])
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'output_ids', 'output_attn_mask', 'labels'])

In [21]:
dataset.set_format(type="torch")
#dataset.set_format(type="torch")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bsize, shuffle=True)

In [22]:
data = next(iter(dataloader))
idx = data["labels"]==0
print(data["input_ids"][idx][0])
print(data["output_ids"][idx][0])

tensor([  464,  1705,  1908, 28289,  7303,   319,   257, 11626,   287,   706,
           12, 24425,  7313,   764, 50258, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257])
tensor([  464,  1705,  1908, 28289,  7303,   319,   257, 11626,   287,   706,
           12, 24425,  7313,   764, 50256, 50257, 50257, 50257, 50257, 50257,
        50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,

## Data Verification

In [23]:
hface_model.to(0)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [24]:
data = next(iter(dataloader))
device = next(hface_model.parameters()).get_device()
with torch.no_grad():
    inpts = {
        "input_ids": data["input_ids"].to(device),
        "attention_mask": data["attention_mask"].to(device),
    }
    outputs = hface_model.transformer(**inpts)
    #logits = outputs.logits
    #print(logits.shape)

In [25]:
outputs.keys()

odict_keys(['last_hidden_state', 'past_key_values'])

In [26]:
outputs["last_hidden_state"].shape

torch.Size([16, 100, 1024])

In [27]:
len(outputs["past_key_values"]) # Number of layers

24

In [28]:
outputs["past_key_values"][0][0].shape # Keys of first layer (or values, I'm not sure which, but probably keys because of naming key_values order)

torch.Size([16, 16, 100, 64])

In [29]:
# model.transformer(inputs_embeds=outputs["last_hidden_state"], attention_mask=inpts["attention_mask"]) # Use this to argue embeddings instead of ids

In [30]:
data["labels"][:10]

tensor([0, 0, 1, 1, 1, 1, 0, 1, 1, 0])

Positive Sample

In [31]:
tokenizer.decode(data["input_ids"][0])

'Nursing schools turned away more than 5,000 qualified applicants in the past year because of shortages of faculty and classroom space.<CLS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [32]:
tokenizer.decode(data["output_ids"][0])

'Nursing schools turned away more than 5,000 qualified applicants in the past year because of shortages of faculty and classroom space.<|endoftext|><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'


Negative Sample

In [33]:
tokenizer.decode(data["input_ids"][1])

'The Java Enterprise System bundles a slew of Sun software for a yearly subscription of $ 100 per employee.<CLS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [34]:
tokenizer.decode(data["output_ids"][1])

'The Java Enterprise System bundles a slew of Sun software for a yearly subscription of $ 100 per employee.<|endoftext|><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

## Model

In [35]:
class SentenceAutoEncoder(torch.nn.Module):
    """
    Trains a new token type to compress a sentence into a single vector representation
    """
    def __init__(self, model, tokenizer, *args, **kwargs):
        """
        model: hugging face transformer model
        """
        super().__init__()
        self.hface_model = model
        self.tokenizer = tokenizer
        self.CLS_ID = tokenizer.cls_token_id
        self.CLS = tokenizer.cls_token
        self.EOS_ID = tokenizer.eos_token_id
        self.EOS = tokenizer.eos_token
        
    def forward(self, data):
        """
        data: dict
            "input_ids": LongTensor (B,S1)
                the token indices of the input sequence. The CLS token should be appended to the end of each sentence.
            "attention_mask": LongTensor (B,S1)
                attention mask for padding purposes. 0s mean padding.
            "output_ids": LongTensor (B,S2)
                the token indices of the target sequence. An EOS token should be appended to the end of each sentence
            "output_attn_mask": LongTensor (B,S2)
                attention mask for padding purposes. 0s mean padding.
        """
        
        model = self.hface_model
        inpt_embs = model.transformer.wte(data["input_ids"]).data
        idx = data["input_ids"]==self.CLS_ID
        inpt_embs[idx] = 0
        inpt_embs[idx] += model.transformer.wte.weight[self.CLS_ID]
        out_embs =  model.transformer.wte(data["output_ids"]).data
        
        
        fx = model.transformer(inputs_embeds=inpt_embs, attention_mask=data["attention_mask"])
        fx = fx["last_hidden_state"][idx][:,None]
        
        
        # Concat compressed representation to beginning of sentence
        attn = torch.cat([torch.ones_like(data["output_attn_mask"][:,:1]), data["output_attn_mask"]], dim=1)
        #attn = torch.pad(data["output_attn_mask"], (1,0))
        try:
            out_embs = torch.cat([fx,out_embs], dim=1)
            attn = torch.cat([torch.ones_like(data["output_attn_mask"][:,:1]), data["output_attn_mask"]], dim=1)
        except:
            print("Data")
            for k in data: print(k, data[k].shape)
            print("idx sum:", idx.float().sum())
            print("In Embds", inpt_embs.shape)
            print("Out Embds", out_embs.shape)
            print("FX", fx.shape)
            assert False
        
        preds = model(inputs_embeds=out_embs, attention_mask=attn).logits
        return preds
        
        
        
        
        

In [36]:
model = SentenceAutoEncoder(hface_model, tokenizer)

## Training

In [37]:
lr = 1e-4
l2 = 1e-3
n_epochs = 50

In [38]:
params = model.hface_model.transformer.wte.parameters()
optimizer = torch.optim.Adam(params, lr=lr, weight_decay=l2)
loss_fxn = torch.nn.CrossEntropyLoss()
params = set(params)
for p in model.parameters():
    if p not in params: p.requires_grad = False

In [ ]:
model.cuda()
for epoch in range(n_epochs):
    print()
    print("Beginning Epoch", epoch)
    model.train()
    avg_loss = 0
    avg_acc = 0
    for i,data in enumerate(dataloader):
        data = {k: v.cuda() for k,v in data.items()}
        preds = model(data)
        preds = preds[:,:-1].reshape(-1, preds.shape[-1])
        labels = data["output_ids"].reshape(-1)
        idx = labels!=tokenizer.pad_token_id
        loss = loss_fxn(preds[idx], labels[idx])
        avg_loss += loss.item()
        argmax = torch.argmax(preds[idx])
        acc = (argmax==labels[idx]).float().mean()
        avg_acc += acc.item()
        if i%10==0: print("Loss", round(loss.item(), 5), "-- Acc:", round(acc.item(), 5))
    print("Avg Loss:", round(avg_loss, 5), "-- Avg Acc:", round(avg_acc, 5))
    print()
        
        


Beginning Epoch 0
Loss 84.24694 -- Acc: 0.0
Loss 81.21325 -- Acc: 0.0
Loss 70.53687 -- Acc: 0.0
Loss 75.70064 -- Acc: 0.0
Loss 89.97944 -- Acc: 0.0
Loss 63.91817 -- Acc: 0.0
Loss 67.60391 -- Acc: 0.0
Loss 79.08193 -- Acc: 0.0
Loss 75.36988 -- Acc: 0.0
Loss 87.33172 -- Acc: 0.0
Loss 78.30166 -- Acc: 0.0
Loss 68.78848 -- Acc: 0.0
Loss 78.52134 -- Acc: 0.0
Loss 69.13879 -- Acc: 0.0
Loss 77.55251 -- Acc: 0.0
Loss 69.66534 -- Acc: 0.0
Loss 84.58016 -- Acc: 0.0
Loss 76.60291 -- Acc: 0.0
Loss 87.75251 -- Acc: 0.0
Loss 82.21988 -- Acc: 0.0
Loss 67.91528 -- Acc: 0.0
Loss 83.62623 -- Acc: 0.0
Loss 93.74638 -- Acc: 0.0
Avg Loss: 17799.98936 -- Avg Acc: 0.0


Beginning Epoch 1
Loss 79.82491 -- Acc: 0.0
Loss 100.64024 -- Acc: 0.0
Loss 70.68429 -- Acc: 0.0
Loss 76.25298 -- Acc: 0.0
Loss 76.36373 -- Acc: 0.0
Loss 75.4203 -- Acc: 0.0
Loss 70.43633 -- Acc: 0.0
Loss 66.94095 -- Acc: 0.0
Loss 60.8101 -- Acc: 0.0
Loss 79.07037 -- Acc: 0.0
Loss 80.07519 -- Acc: 0.0
Loss 74.47204 -- Acc: 0.0
Loss 64.73815 